# Turkish Diacritisation | YZV 405E NLP Term Project

Author: Bora Boyacıoğlu

Student ID: 150200310

## Step 3: Evaluating

**Note:** This notebook is not designed for Google Colab.

In [1]:
%pip install unidecode --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import csv
import random
import sys

import pickle as pkl
import torch

In [3]:
%load_ext autoreload
%autoreload 2

### Reloading the Processed Data

In [4]:
path = ''

Mount the drive for Google Colab. <font color='red'>Do not run this for local use.</font>

`Command + Shift + 7` to comment out or uncomment.

In [5]:
# # Mound the Drive.
# from google.colab import drive
# drive.mount('/content/drive')

# # Delete the sample_data folder because I don't like unnecessary things.
# !rm -rf sample_data

# # Update the data folder.
# path = '/content/drive/MyDrive/Share/NLP/'

# # Append the data folder path to system.
# sys.path.append(path)

Import local libraries and classes. Open data and vocab files.

In [6]:
from utils.model import Encoder, Decoder, Seq2Seq
from utils.utils import *

# Load the test data.
with open(path + 'data/test_data.pkl', 'rb') as f:
    test_data = pkl.load(f)

# Load the vocabulary.
with open(path + 'data/vocab.pkl', 'rb') as f:
    vocab = pkl.load(f)

length = len(test_data)

### Reloading the Model

In [7]:
# Define the device.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the parameters (same as the ones used for training).
params = {
    'emb_dim': 64,
    'hid_dim': 256,
    'n_layers': 2,
    'dropout': 0.5
}

# Initialize the model.
encoder = Encoder(input_dim=len(vocab['w2i']), **params)
decoder = Decoder(output_dim=len(vocab['w2i']) - 1, **params)
model = Seq2Seq(encoder, decoder, device).to(device)

# Load the model's state_dict.
checkpoint = torch.load(path + 'models/e50-l3.89-p64_256_2_0.5_18.pth', map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(145437, 64)
    (rnn): LSTM(64, 256, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(145436, 64)
    (rnn): LSTM(64, 256, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=256, out_features=145436, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

Defining the prediction function.

In [8]:
def predict(model, sentence, vocab, device):
    max_len = len(sentence)
        
    # Convert sentence to tensor and add a batch dimension.
    sentence_tensor = torch.tensor(sentence, dtype=torch.long).unsqueeze(1).to(device)
    
    # Initialize.
    with torch.no_grad():
        hidden, cell = model.encoder(sentence_tensor)

    outputs = []
    input = sentence_tensor[0,:]  # Start token, typically.
    
    # Generate sequence.
    for _ in range(max_len):
        
        with torch.no_grad():
            output, hidden, cell = model.decoder(input, hidden, cell)
            
            # Get the most probable next word index.
            top1 = output.argmax(1)
        
        # Check for end of sentence token.
        if top1.item() == vocab['w2i']['<eos>']:
            break
        
        # Update input for next iteration.
        input = top1
        outputs.append(top1.item())
    
    # Convert indices to words.
    translated_sentence = [vocab['i2w'][idx] for idx in outputs]
    return ' '.join(translated_sentence)

Making a random prediction from the test data.

In [9]:
index = random.randint(0, length - 1)
sentence = test_data.get(index, 'und')
actual = untokenize(test_data, index, 'und')

prediction = predict(model, sentence, vocab, device)
print(f"Predicted ({len(prediction.split(' '))}):", prediction)
print(f"Actual    ({len(actual.split(' '))}):", actual)

Predicted (1): yazar
Actual    (22): yazar tayfun atay bikkinlik vermis bir yesilcam klisesine yaslanma riskini basariyla asan zengin kiz fakir oglan erkeklik uzerine vurgulariyla izlenmeyi hak ediyor


### Predicting

Here, the entire test data will be predicted using the model.

In [10]:
predictions = []

for i in range(length):
    # Print the progress.
    print(f'\rPredicting {100 * (i + 1) / length:.2f}%', end='')
    
    # Get the sentence.
    sentence = test_data.get(i, 'und')
    
    # Make the prediction.
    prediction = predict(model, sentence, vocab, device)
    
    # Append the prediction.
    predictions.append(prediction)

Predicting 100.00%

Write the predictions to a file.

In [11]:
save_file = path + 'data/comb/predictions.csv'
max_rows = 1156

with open(save_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ID', 'Sentence'])
    
    for i, sentence in enumerate(predictions):
        if i > max_rows:
            break
        
        if not sentence:
            sentence = ' '
        
        writer.writerow([i, sentence])

### Get the Scores

By comparing the results with the golden test data, get the score points.

In [23]:
get_scores(predictions[:max_rows], path + 'data/test_gold.csv');

Word Score:	2.60%
Sentence Score:	0.52%
